# Evaluating Bethe variational inference

The results from this notebook were generated by running
```sh
python bethe_vi.py
```

In [ ]:
import torch
import matplotlib.pyplot as plt

In [ ]:
results = torch.load("results/bethe_vi.pt")
print("results:", list(results))
data = results["data"]
samples = results["samples"]
print("data:", list(data))
print("samples:", list(samples))

In [ ]:
sample_id = 0
tree = samples["trees"][sample_id]
code = samples["codes"][sample_id]
fig, axes = plt.subplots(2, 2, figsize=(8, 8), sharex="col", sharey="row", dpi=300)

def plot(ax, X, Y):
    x, y = [], []
    for i, j in enumerate(tree.parents.tolist()):
        if j != -1:
            x.extend((X[i], X[j], None))
            y.extend((Y[i], Y[j], None))
    ax.plot(x, y, 'b-', lw=0.5)
    ax.scatter(X, Y, s=3, color="blue")
    ax.scatter(X[tree.leaves], Y[tree.leaves], s=10, color="black", zorder=100)

plot(axes[0][1], code[:, 1], code[:, 0])
plot(axes[0][0], tree.times, code[:, 0])
plot(axes[1][1], code[:, 1], tree.times)
plt.subplots_adjust(hspace=0, wspace=0)